In [7]:
import librosa 
import numpy as np
import os
import matplotlib.pyplot as plt
from typing import List, Tuple
import scipy

# Plotting All Strings

In [ ]:
def plot_spectrum(signal, name, sr, max = 500):
    freqs, magnitudes = get_spectrum(signal, sr)
    # Discard negative frequencies
    freqs = freqs[:len(freqs) // 2]
    magnitudes = magnitudes[:len(magnitudes) // 2]
    
    norm_magnitudes = librosa.util.normalize(magnitudes)
    
    N = len(norm_magnitudes)
    # Plot the FFT frequencies vs. norm_magnitudes
    plt.figure(figsize=(10, 6))
    plt.plot(freqs[:N // 2], norm_magnitudes[:N // 2])  # Plot the positive frequencies
    plt.xlim(0, max)
    plt.title("FFT of " + name)
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Magnitude")
    plt.grid(True)
    plt.show()
    # Find the index of the maximum magnitude
    fundamental_freq = f0(freqs, norm_magnitudes, sr)
    print("Frequency:", fundamental_freq)
    
    print("Note: ", librosa.hz_to_note(fundamental_freq))

def get_spectrum(signal, sr) -> Tuple[np.ndarray, np.ndarray]:
    """ 
    Returns:
        - freqs: the frequencies corresponding to the magnitudes
        - magnitudes: the magnitudes of the FFT
    """
    fhat = np.fft.fft(signal)
    magnitudes = np.abs(fhat)

    # Calculate the corresponding frequencies
    N = len(magnitudes)
    freqs = np.fft.fftfreq(N, 1.0 / sr)
    return freqs, magnitudes

def f0(freqs, norm_magnitudes, sr) -> np.ndarray:
    # Find a set of possible fundamental frequencies by finding peaks in the spectrum
    fundamental_freqs = scipy.signal.find_peaks(norm_magnitudes, height=0.2, threshold=0.2)[0]
    N = len(norm_magnitudes)
    
    hps = np.ones(len(fundamental_freqs))
    for i in range(len(fundamental_freqs)):
        for r in range(2, 9):
            if fundamental_freqs[i] * r >= N:
                break
            hps[i] *= norm_magnitudes[fundamental_freqs[i] * r]
    
            
    return freqs[fundamental_freqs[np.argmax(hps)]]

files = os.listdir("recordings_wav")
for file in files:
    signal, sr = librosa.load("recordings_wav/" + file)
    plot_spectrum(signal, file, sr)
        

# RT note Identification

- Listen to the audio stream
- Sample the audio stream
- analyze the samples audio

In [9]:
import sounddevice as sd
from scipy.io.wavfile import write
import noisereduce as nr

In [ ]:
# def sample(sr, time) -> np.array:
#     recording = sd.rec(int(time * sr), samplerate=sr, channels=1, dtype=np.float64)
#     sd.wait()
#     # write to wav file
#     write("sample.wav", sr, recording)
#     load, sr = librosa.load("sample.wav")
#     reduced_noise = nr.reduce_noise(y=load, sr= sr)
#     write("sample.wav", sr, reduced_noise)
    
# sample(22050, 4)

signal, sr = librosa.load("sample.wav")
librosa.display.waveshow(signal, sr=sr)
onsets = librosa.onset.onset_detect(y=signal, sr=sr)
times = librosa.frames_to_time(onsets)
# plot the onsets
plt.vlines(times, -1, 1, color="r")

note = signal
note_db = librosa.amplitude_to_db(note)

plot_spectrum(signal, "sample", sr)
    
